# ResNet
---
ResNet（Residual Network）是由微软研究院的何恺明等人在2015年提出的一种深度卷积神经网络架构，它在当年的ImageNet大规模视觉识别挑战赛（ILSVRC）中取得了优异的成绩，赢得了冠军。ResNet的核心创新是引入了残差学习（Residual Learning）的概念，解决了深度神经网络在训练过程中遇到的梯度消失和梯度爆炸问题，从而使得训练非常深的网络成为可能。

## 主要特点
1. 残差学习：
    ResNet的核心思想是通过引入残差块（Residual Block）来实现残差学习。残差块的基本结构是将输入数据直接添加到网络的输出中，从而形成一个残差映射（Residual Mapping）。具体来说，假设输入为 `x`，网络的输出为`F(x)`，那么残差块的输出为`H(x) = F(x) + x`。这种设计使得网络可以学习到输入和输出之间的残差，而不是直接学习输出。

    - 残差块的结构：

        - 输入 x 经过一系列卷积层（通常包括卷积、批量归一化和激活函数）得到 F(x)。

        - 将 F(x) 与输入 x 相加，得到残差块的输出 H(x) = F(x) + x。

        - 最后，对 H(x) 进行激活函数处理（通常是ReLU）。

2. 跳跃连接（Skip Connection）：
    ResNet通过跳跃连接（Skip Connection）实现了残差学习。跳跃连接将输入数据直接传递到网络的输出，从而避免了梯度消失问题。跳跃连接使得网络可以更容易地学习到恒等映射（Identity Mapping），即 F(x) = 0，从而简化了网络的学习过程。

3. 深度网络：
    ResNet通过堆叠多个残差块来构建非常深的网络。常见的ResNet架构包括ResNet-18、ResNet-34、ResNet-50、ResNet-101和ResNet-152，其中数字表示网络的层数。ResNet-152是当时最深的网络之一，达到了152层。

4. 批量归一化：
    ResNet在每个卷积层之后使用了批量归一化（Batch Normalization），这有助于加速训练过程并提高模型的泛化能力。



## 残差块

![res](https://zh-v2.d2l.ai/_images/residual-block.svg "res")

## 包含与不包含1×1卷积层的残差块

![Res](https://zh-v2.d2l.ai/_images/resnet-block.svg "Res")

## 网格结构
1. 输入层：
    - 输入层接收224x224x3的图像数据。

2. 卷积层和池化层：
    - 在进入残差块之前，网络首先通过几个卷积层和池化层来提取基本的特征。

3. 残差块：
    - 网络的主体由多个残差块组成，每个残差块内部包含多个卷积层和跳跃连接。

4. 全局平均池化层：
    - 在网络的最后，使用全局平均池化层将特征图转换为一个固定长度的向量。

5. 全连接层和softmax层：
    - 最终的全连接层和softmax层用于输出分类结果。

## 流程图

![Process](https://zh-v2.d2l.ai/_images/resnet18.svg "Processing")

## 简单代码实现


In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l


class Residual(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [3]:
blk = Residual(3,3)
X = torch.rand(4, 3, 6, 6)
Y = blk(X)
Y.shape

torch.Size([4, 3, 6, 6])

In [4]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [5]:
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [6]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

In [7]:
net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 10))

In [8]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])
